In [ ]:
#!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calc_angle(a,b,c): # angle between 3 points
    a = np.array(a) # first joint
    b = np.array(b) # mid joint
    c = np.array(c) # end joint
    
    radian = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radian*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

In [3]:
def squat_counter():
    
    counter = 0
    stage = "-" # down position or up position
    
    # EXTRACT LANDMARKS
    try: 
        landmarks = results.pose_landmarks.landmark

        # GET COORDINATES
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        left_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]

        # CALCULATE ANGLE 
        angle = calc_angle(left_hip, left_knee, left_heel)
        #print(angle)

        # VISUALIZE ANGLE
        cv2.putText(image,str(round(angle,2)),
                    # determine position to display text of angle
                    tuple(np.multiply(left_knee,[640,480]).astype(int)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5,  (255, 255, 255), 2, cv2.LINE_AA 
                   )

        # SQUAT LOGIC
        if angle > 170:
            stage = "up"
        if angle < 120 and stage == "up":
            stage = "down"
            counter += 1
            print(counter)
    
        return counter, stage

    except: 
        pass


In [5]:
cap = cv2.VideoCapture(0) 

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: # accuracy 
    while cap.isOpened():
        ret,frame = cap.read() 
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
        image.flags.writeable = False 
        
        results = pose.process(image) 
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
        
        counter, stage = squat_counter()

        # RENDER COUNTER
        # Setup status box
        cv2.rectangle(image, (0,0), (235,73), (245,117,16), -1) 
        
        # Rep data
        cv2.putText(image, "REPS:", (15,12), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) # font, size, color, line width, line type
        
        cv2.putText(image, str(counter), (10,70),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, "STAGE:", (100,12), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) 
        
        cv2.putText(image, stage, (110,70),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
            
  
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(0,0,255),thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(0,255,0),thickness=2, circle_radius=2) 
                                 )
        
        window_name = 'Exercise Counter'
        
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.imshow(window_name, image)

        if cv2.waitKey(10) & 0xFF == ord("q"): 
            break

    cap.release() 
    cv2.destroyAllWindows()